In [1]:
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sentence_transformers import SentenceTransformer
from sentence_transformers import util
from flair.data import Sentence
from flair.models import SequenceTagger
from scipy.optimize import linear_sum_assignment
import pandas as pd
import numpy as np
embedder_model = SentenceTransformer('all-MiniLM-L6-v2')
tagger = SequenceTagger.load("flair/chunk-english")

/Users/elantonfernandes/opt/anaconda3/envs/modAL_test/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2022-06-20 09:51:52,151 loading file /Users/elantonfernandes/.flair/models/chunk-english/5b53097d6763734ee8ace8de92db67a1ee2528d5df9c6d20ec8e3e6f6470b423.d81b7fd7a38422f2dbf40f6449b1c63d5ae5b959863aa0c2c1ce9116902e8b22
2022-06-20 09:51:53,064 SequenceTagger predicts: Dictionary with 45 tags: <unk>, O, B-NP, E-NP, I-NP, S-PP, S-VP, S-SBAR, S-ADVP, S-NP, S-ADJP, B-VP, E-VP, B-PP, E-PP, I-VP, S-PRT, B-ADVP, E-ADVP, B-ADJP, E-ADJP, B-CONJP, I-CONJP, E-CONJP, I-ADJP, B-SBAR, E-SBAR, S-INTJ, I-ADVP, I-PP, B-UCP, I-UCP, E-UCP, S-LST, B-PRT, I-PRT, E-PRT, S-CONJP, B-INTJ, E-INTJ, I-INTJ, B-LST, E-LST, <START>, <STOP>


In [2]:
learner1 = ActiveLearner(
    estimator=RandomForestClassifier(),
    query_strategy=uncertainty_sampling
)

learner2 = ActiveLearner(
    estimator=RandomForestClassifier(),
    query_strategy=uncertainty_sampling
)

learner3 = ActiveLearner(
    estimator=RandomForestClassifier(),
    query_strategy=uncertainty_sampling
)

learner4 = ActiveLearner(
    estimator=RandomForestClassifier(),
    query_strategy=uncertainty_sampling
)

In [3]:
dataframe = pd.read_csv('/Users/elantonfernandes/Development/HBRS/Untitled Folder/uncertainty_scores.csv')

In [4]:
dataframe[10:30]

,Answer ID,Answer Index,Query Answer,Student Answer,Average Score,Uncertainty Score,Sampled
10,1.1,10,To simulate the behaviour of portions of the d...,A program that stimulates the behavior of port...,5.0,0.739017,No
11,1.1,11,To simulate the behaviour of portions of the d...,A program that simulates the behavior of porti...,5.0,0.882844,No
12,1.1,12,To simulate the behaviour of portions of the d...,To lay out the basics and give you a starting ...,2.0,0.383955,No
13,1.1,13,To simulate the behaviour of portions of the d...,To simulate problem solving for parts of the p...,4.5,0.565983,No
14,1.1,14,To simulate the behaviour of portions of the d...,A prototype program provides a basic groundwor...,2.0,0.475071,No
15,1.1,15,To simulate the behaviour of portions of the d...,A prototype program is a part of the Specifica...,4.5,0.445633,No
16,1.1,16,To simulate the behaviour of portions of the d...,Program that simulates the behavior of portion...,5.0,0.836386,No
17,1.1,17,To simulate the behaviour of portions of the d...,it provides a limited proof of concept to veri...,2.0,0.491766,No
18,1.1,18,To simulate the behaviour of portions of the d...,It tests the main function of the program whil...,2.0,0.504266,No
19,1.1,19,To simulate the behaviour of portions of the d...,To get early feedback from users in early stag...,2.5,0.452951,No


### Stratergy 1:
Simple technique: 
* Direct sentence comparison and addition of feedback to the whole answer.
* Sample student answers based on the uncertainty score (lowest first)
* The number of samples is decided by the percentage set by the oracle.
* Design the extraction based on pool based active learning to easily delete samples after they have been used for training.
* 

In [5]:
def uncertaintySampling(data,q_id,per=.30):
        
    df = data.loc[(data['Answer ID']==q_id)]

    oracle_samples = df.nsmallest(int(len(df.index)*per),columns="Uncertainty Score")

    #present the samples to the oracle and ask them to grade the answers based on the 
    #chunks and identify which chunks contributed to the answer.

    return oracle_samples

In [6]:
data_samples = uncertaintySampling(dataframe,q_id=1.1)

In [7]:
data_samples

,Answer ID,Answer Index,Query Answer,Student Answer,Average Score,Uncertainty Score,Sampled
5,1.1,5,To simulate the behaviour of portions of the d...,To find problem and errors in a program before...,2.0,0.261266,No
28,1.1,28,To simulate the behaviour of portions of the d...,Prototype programming is an approach to progra...,2.5,0.355348,No
23,1.1,23,To simulate the behaviour of portions of the d...,To ease the understanding of problem under dis...,2.5,0.375454,No
12,1.1,12,To simulate the behaviour of portions of the d...,To lay out the basics and give you a starting ...,2.0,0.383955,No
15,1.1,15,To simulate the behaviour of portions of the d...,A prototype program is a part of the Specifica...,4.5,0.445633,No
8,1.1,8,To simulate the behaviour of portions of the d...,-To provide an example or model of how the fin...,3.5,0.447838,No
19,1.1,19,To simulate the behaviour of portions of the d...,To get early feedback from users in early stag...,2.5,0.452951,No
3,1.1,3,To simulate the behaviour of portions of the d...,Defined in the Specification phase a prototype...,5.0,0.456788,No


In [8]:
feedbacks_s1 = ['correct','missing keywords','interchanged terms']

In [9]:
for row_index, row_value in data_samples.iterrows():
    
    query_answer = row_value['Query Answer']
    print("Model answer:\n",query_answer)
    student_answer = row_value['Student Answer']
    print("Student answer:\n",student_answer)
    student_answer_embedding = embedder_model.encode(student_answer)
    #print("Student answer Embedding",student_answer_embedding.reshape(1,-1))
    print("What feedback would you give this answer?")
    for i,f in enumerate(feedbacks_s1):
        print(i,":",f)
    feedback_option_s1 = input("\n0: Add from existing feedback.\n1: Add new feedback.")
    if feedback_option_s1 == '0':
        feedback_id_s1 = np.array([int(input("\nSelect feedback ID"))], dtype=int)
    elif feedback_option_s1 == '1':
        feedback_statement_s1 = input("\nEnter new feedback")

        feedbacks_s1.append(feedback_statement_s1)
        feedback_id_s1 = np.array([feedbacks_s1.index(feedback_statement_s1)], dtype=int)
        print("feedback id",feedback_id_s1)
    else:
        print("\nWrong id selected")
    #print("Student answer shape",student_answer_embedding.shape)
    #print("Feedback id embedding",feedback_id.shape)
    learner1.teach(student_answer_embedding.reshape(1,-1),feedback_id_s1)

Model answer:
 To simulate the behaviour of portions of the desired software product. 
Student answer:
 To find problem and errors in a program before it is finalized
What feedback would you give this answer?
0 : correct
1 : missing keywords
2 : interchanged terms

0: Add from existing feedback.
1: Add new feedback.0

Select feedback ID0
Model answer:
 To simulate the behaviour of portions of the desired software product. 
Student answer:
 Prototype programming is an approach to programming that enables one to take an organized approach to developing an effective program with minimal errors and a strategic pattern when solving a problem. i.e. book gave an example of a costumer withdrawing money from a bank, the approach that was taking on a pseudo code level during the OOA/Design lvl before proceeding into creating a solution. 
What feedback would you give this answer?
0 : correct
1 : missing keywords
2 : interchanged terms

0: Add from existing feedback.
1: Add new feedback.0

Select 

### Prediction

In [10]:
for row_index, row_value in dataframe.iterrows():
    if row_index <= 28:
        
        query_answer = row_value['Query Answer']
        print("\n==========================================")
        print("Model answer:\n",query_answer)
        student_answer = row_value['Student Answer']
        print("Student answer:\n",student_answer)
        student_answer_embedding = embedder_model.encode(student_answer)
        #print("Student answer Embedding",student_answer_embedding.reshape(1,-1))
        
        feedback = learner1.predict(student_answer_embedding.reshape(1,-1))
        print("Feedback id:",feedback)
        print("Corresponding feedback\n",feedbacks_s1[feedback[0]])
    else:
        pass


Model answer:
 To simulate the behaviour of portions of the desired software product. 
Student answer:
 High risk problems are address in the prototype program to make sure that the program is feasible.  A prototype may also be used to show a company that the software can be possibly programmed.  
Feedback id: [0]
Corresponding feedback
 correct

Model answer:
 To simulate the behaviour of portions of the desired software product. 
Student answer:
 To simulate portions of the desired final product with a quick and easy program that does a small specific job. It is a way to help see what the problem is and how you may solve it in the final project.
Feedback id: [0]
Corresponding feedback
 correct

Model answer:
 To simulate the behaviour of portions of the desired software product. 
Student answer:
 A prototype program simulates the behaviors of portions of the desired software product to allow for error checking.
Feedback id: [0]
Corresponding feedback
 correct

Model answer:
 To simu

##### Stratergy 2:
* break down the answer into its chunks.
* Ask the oracle to form facts with the given chunks
* Add feedback to the fact and a grade and save this to a different .csv file

In [11]:
feedbacks_s2 = ['correct','missing keywords','interchanged terms']

In [12]:

fact_dataframe = pd.DataFrame(columns=['Fact','Score','Feedback'])

In [13]:
fact_dataframe

,Fact,Score,Feedback


In [14]:
for row_index, row_value in data_samples.iterrows():
    
    query_answer = row_value['Query Answer']
    print("Model answer:\n",query_answer)

    query_chunker = Sentence(query_answer)
    tagger.predict(query_chunker)

    student_answer = row_value['Student Answer']
    print("Student answer:\n",student_answer)
    
    passage_chunker = Sentence(student_answer)
    tagger.predict(passage_chunker)
    
    for ind,chunk in enumerate(passage_chunker.get_spans('np')):
        print(ind,":",chunk.text)
        
    fact_list = list()
    feedback_id_list = list()
    score_list = list()
    print("Select chunks to form facts from the above list")
    
    while(True):
        chunk_list = list()
        fact_create = int(input("Do you wish to create a fact? y:1/n:0?"))
        print(fact_create)
        if fact_create:
                        
            while(True):
                inp = input("Enter the chunk id related to one fact or x to exit")
                if inp == 'x':
                    fact_list.append(chunk_list)
                    break
                else:
                    chunk_list.append(passage_chunker.get_spans('np')[int(inp)].text)

            print("What feedback would you give this answer?")
            for i,f in enumerate(feedbacks_s2):
                print(i,":",f)
            
            feedback_option_s2 = input("\n0: Add from existing feedback.\n1: Add new feedback.")
            if feedback_option_s2 == '0':
                feedback_id_s2 = int(input("\nSelect feedback ID"))
                print("feedback id",feedback_id_s2)
                feedback_id_list.append(feedback_id_s2)
            elif feedback_option_s2 == '1':
                feedback_statement_s2 = input("\nEnter new feedback")

                feedbacks_s2.append(feedback_statement_s2)
                feedback_id_s2 = int(feedbacks_s2.index(feedback_statement_s2))
                print("feedback id",feedback_id_s2)
                feedback_id_list.append(feedback_id_s2)
            else:
                print("\nWrong id selected")
                
            print("\nWhat score contribution would you associate with this fact?")
            score = input("\nEnter score:\n")
            score_list.append(score)
            
        else:
            break
            
    print("\nFact list",fact_list)
    print("\nFeedback list",feedback_id_list)
    print("\nScore list",score_list)
    if len(fact_list)!=0:
        for fact_entry,feedback_entry, fact_score in zip(fact_list,feedback_id_list,score_list):
            text = ' '.join(fact_entry)
            fback = feedbacks_s2[feedback_entry]
            data = [[text,score,fback]]
            print(data)
            
            temp = pd.DataFrame(
            data = data,columns=['Fact','Score','Feedback'])
            fact_dataframe = pd.concat([fact_dataframe,temp],ignore_index=True)
            fact_dataframe.to_csv('facts_new.csv',index=False)  

Model answer:
 To simulate the behaviour of portions of the desired software product. 
Student answer:
 To find problem and errors in a program before it is finalized
0 : To find
1 : problem and errors
2 : in
3 : a program
4 : before
5 : it
6 : is finalized
Select chunks to form facts from the above list
Do you wish to create a fact? y:1/n:0?1
1
Enter the chunk id related to one fact or x to exit0
Enter the chunk id related to one fact or x to exit1
Enter the chunk id related to one fact or x to exit2
Enter the chunk id related to one fact or x to exit3
Enter the chunk id related to one fact or x to exit4
Enter the chunk id related to one fact or x to exit5
Enter the chunk id related to one fact or x to exit6
Enter the chunk id related to one fact or x to exitx
What feedback would you give this answer?
0 : correct
1 : missing keywords
2 : interchanged terms

0: Add from existing feedback.
1: Add new feedback.0

Select feedback ID1
feedback id 1

What score contribution would you associ

Enter the chunk id related to one fact or x to exit4
Enter the chunk id related to one fact or x to exitx
What feedback would you give this answer?
0 : correct
1 : missing keywords
2 : interchanged terms

0: Add from existing feedback.
1: Add new feedback.0

Select feedback ID0
feedback id 0

What score contribution would you associate with this fact?

Enter score:
0.5
Do you wish to create a fact? y:1/n:0?1
1
Enter the chunk id related to one fact or x to exit0
Enter the chunk id related to one fact or x to exit8
Enter the chunk id related to one fact or x to exit9
Enter the chunk id related to one fact or x to exit10
Enter the chunk id related to one fact or x to exit11
Enter the chunk id related to one fact or x to exit12
Enter the chunk id related to one fact or x to exitx
What feedback would you give this answer?
0 : correct
1 : missing keywords
2 : interchanged terms

0: Add from existing feedback.
1: Add new feedback.0

Select feedback ID0
feedback id 0

What score contribution 

#  learning
* active learner learn the individual facts for the feedback and score

In [15]:
fact_dataframe = pd.read_csv('facts.csv')
fact_dataframe

,Fact,Score,Feedback
0,To find problem and errors in a program before...,0.5,missing keywords
1,Prototype programming enables to take an organ...,1.0,correct
2,To ease the understanding of problem under dis...,0.5,missing keywords
3,to ease the understanding of the program,0.5,missing keywords
4,A prototype program 's employed to illustrate ...,1.0,correct
5,To provide an example or model of how the fini...,1.0,correct
6,To show users a first idea of what the program...,1.0,correct
7,a prototype stimulates the behavior of portion...,1.0,correct


In [16]:
score_list = [0.5,1,-1,-0.5]

In [17]:
for row_id,row_value in fact_dataframe.iterrows():
    fact_embedding = embedder_model.encode(row_value['Fact'])
    feedback_index = np.array([feedbacks_s2.index(row_value['Feedback'])],dtype=int)
    #print("\n",type(feedback_index))
    score_index = np.array([score_list.index(row_value['Score'])],dtype=int)
    #print(type(score_index))
    learner3.teach(fact_embedding.reshape(1,-1),feedback_index)
    learner4.teach(fact_embedding.reshape(1,-1),score_index)

###### Prediction 2.1
* model and student answer are made into chunks.
* linear sum asssignemnt is applied to these chunks to find the best possible pairs.
* The matching chunks from student answer are converted into a sentence again.
* This sentence is used in the active learner prediction.

In [18]:
def get_chunk_similarity_score_matrix(query_chunks,passage_chunks):

    similarity_score_matrix = np.zeros((len(query_chunks),len(passage_chunks)))

    for i,query_entity in enumerate(query_chunks):
        for j,passage_entity in enumerate(passage_chunks):
            similarity_score_matrix[i][j] = util.dot_score(embedder_model.encode(query_entity.text), 
                                                          embedder_model.encode(passage_entity.text))
           
        """ 
           if threshold >= 0.7:
                similarity_score_matrix[i][j]= threshold
            else:
                similarity_score_matrix[i][j] = 0
"""

    return similarity_score_matrix

def apply_linear_sum_assignment(similarity_score_matrix):
    
    row_ind, col_ind = linear_sum_assignment(similarity_score_matrix,maximize=True)
    return (row_ind,col_ind)

In [25]:

for row_index, row_value in dataframe.iterrows():
    if row_index <= 28:
        
        query_answer = row_value['Query Answer']
        query_chunker = Sentence(query_answer)
        tagger.predict(query_chunker)
        
        student_answer = row_value['Student Answer']
        passage_chunker = Sentence(student_answer)
        tagger.predict(passage_chunker)
        
        similarity_score_matrix = get_chunk_similarity_score_matrix(
            query_chunker.get_spans('np'),
            passage_chunker.get_spans('np'))
        
        row_ind, col_ind = linear_sum_assignment(similarity_score_matrix)
        #print(type(row_ind), col_ind)
        #row_ind = row_ind.sort()
        #col_ind = col_ind.sort()
        query_text = ' '.join([query_chunker.get_spans('np')[x].text for x in row_ind])
        passage_text = ' '.join([passage_chunker.get_spans('np')[x].text for x in col_ind])
        print("\n===========================================================================")
        print("\nBefore linear sum assignment")
        print("\nQuery text:\n",query_answer)
        print("\nPassage text:\n",student_answer)
        print("\nAfter linear sum assignment.")
        print("\nQuery text:\n",query_text)
        print("\nPassage text:\n",passage_text)
        
        text_embed = embedder_model.encode(passage_text)
        feedback_ind = learner3.predict(text_embed.reshape(1,-1))
        #print(feedback_ind[0])
        print("Feedback given: ",feedbacks_s2[feedback_ind[0]])
        score_ind = learner4.predict(text_embed.reshape(1,-1))
        print("Score given: ",score_list[score_ind[0]])
        
        """student_answer_embedding = embedder_model.encode(student_answer)
        #print("Student answer Embedding",student_answer_embedding.reshape(1,-1))
        
        feedback = learner1.predict(student_answer_embedding.reshape(1,-1))
        print("Feedback id:",feedback)
        print("Corresponding feedback\n",feedbacks_s1[feedback[0]])"""
    else:
        pass



Before linear sum assignment

Query text:
 To simulate the behaviour of portions of the desired software product. 

Passage text:
 High risk problems are address in the prototype program to make sure that the program is feasible.  A prototype may also be used to show a company that the software can be possibly programmed.  

After linear sum assignment.

Query text:
 To simulate the behaviour of portions of the desired software product

Passage text:
 a company may also be used can be possibly programmed High risk problems the prototype program sure
Feedback given:  correct
Score given:  1


Before linear sum assignment

Query text:
 To simulate the behaviour of portions of the desired software product. 

Passage text:
 To simulate portions of the desired final product with a quick and easy program that does a small specific job. It is a way to help see what the problem is and how you may solve it in the final project.

After linear sum assignment.

Query text:
 To simulate the behav



Before linear sum assignment

Query text:
 To simulate the behaviour of portions of the desired software product. 

Passage text:
 A prototype program provides a basic groundwork from which to further enhance and improve a solution to a problem.

After linear sum assignment.

Query text:
 To simulate the behaviour of portions of the desired software product

Passage text:
 which to further enhance A prototype program to a basic groundwork improve
Feedback given:  correct
Score given:  1


Before linear sum assignment

Query text:
 To simulate the behaviour of portions of the desired software product. 

Passage text:
 A prototype program is a part of the Specification phase of Software Problem Solvin.  It's employed to illustrate how the key problem or problems will be solved in a program, and sometimes serves as a base program to expand upon.

After linear sum assignment.

Query text:
 To simulate the behaviour of portions of the desired software product

Passage text:
 the key probl

###### Prediction 2.2
* student answer in converted into its individual sentences.
* ench sentence is passed through the active learner to get the individual feedback and score.


In [26]:
from nltk.tokenize import sent_tokenize
"""token_text = sent_tokenize(text)
print("\nSentence-tokenized copy in a list:")
print(token_text)
print("\nRead the list:")
for s in token_text:
    print(s)
  """  

for row_index, row_value in dataframe.iterrows():
    if row_index <= 28:
        
        student_answer = row_value['Student Answer']
        print("\n=============================================================")
        print("Student answer",student_answer)
        student_answer_sents = sent_tokenize(student_answer)
        
        feedback_score_pairs = list()
        for i,sent in enumerate(student_answer_sents):
            sent_embed = embedder_model.encode(sent)
            feedback_ind = learner3.predict(sent_embed.reshape(1,-1))
            score_ind = learner4.predict(sent_embed.reshape(1,-1))
            feedback_score_pairs.append([feedbacks_s2[feedback_ind[0]],score_list[score_ind[0]]])
            print("\nAssociated feedback(s) for sentence: ",i+1," : ",feedbacks_s2[feedback_ind[0]],
                  "\n Associated score",score_list[score_ind[0]])
    else:
        pass
    


Student answer High risk problems are address in the prototype program to make sure that the program is feasible.  A prototype may also be used to show a company that the software can be possibly programmed.  

Associated feedback(s) for sentence:  1  :  correct 
 Associated score 1

Associated feedback(s) for sentence:  2  :  correct 
 Associated score 1

Student answer To simulate portions of the desired final product with a quick and easy program that does a small specific job. It is a way to help see what the problem is and how you may solve it in the final project.

Associated feedback(s) for sentence:  1  :  correct 
 Associated score 1

Associated feedback(s) for sentence:  2  :  missing keywords 
 Associated score 0.5

Student answer A prototype program simulates the behaviors of portions of the desired software product to allow for error checking.

Associated feedback(s) for sentence:  1  :  correct 
 Associated score 1

Student answer Defined in the Specification phase a pro


Associated feedback(s) for sentence:  3  :  missing keywords 
 Associated score 1


In [ ]:
import numpy as np

from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from IPython import display
from matplotlib import pyplot as plt
%matplotlib inline

n_initial = 100

X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

initial_idx = np.random.choice(range(len(X_train)), size=n_initial, replace=False)

X_initial, y_initial = X_train[initial_idx], y_train[initial_idx]
X_pool, y_pool = np.delete(X_train, initial_idx, axis=0), np.delete(y_train, initial_idx, axis=0)

learner = ActiveLearner(
    estimator=RandomForestClassifier(),
    query_strategy=uncertainty_sampling,
    X_training=X_initial, y_training=y_initial
)



n_queries = 20


accuracy_scores = [learner.score(X_test, y_test)]

for i in range(n_queries):
    display.clear_output(wait=True)
    query_idx, query_inst = learner.query(X_pool)
    with plt.style.context('seaborn-white'):
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.title('Digit to label')
        plt.imshow(query_inst.reshape(8, 8))
        plt.subplot(1, 2, 2)
        plt.title('Accuracy of your model')
        plt.plot(range(i+1), accuracy_scores)
        plt.scatter(range(i+1), accuracy_scores)
        plt.xlabel('number of queries')
        plt.ylabel('accuracy')
        display.display(plt.gcf())
        plt.close('all')

    print("Which digit is this?")
    y_new = np.array([int(input())], dtype=int)
    learner.teach(query_inst.reshape(1, -1), y_new)
    X_pool, y_pool = np.delete(X_pool, query_idx, axis=0), np.delete(y_pool, query_idx, axis=0)
    accuracy_scores.append(learner.score(X_test, y_test))
    
    
with plt.style.context('seaborn-white'):
    plt.figure(figsize=(10, 5))
    plt.title('Accuracy of the classifier during the active learning')
    plt.plot(range(n_queries+1), accuracy_scores)
    plt.scatter(range(n_queries+1), accuracy_scores)
    plt.xlabel('number of queries')
    plt.ylabel('accuracy')
    plt.show()